# 加载所有数据

In [2]:
import limap
import limap.util.io as limapio
import matplotlib.pyplot as plt
import numpy as np
import plyfile
import torch
import tqdm

# 加载线
lines_path = '../../datasets/jmsd/perspective_v1/lines/finaltracks/'
lines, linetracks = limapio.read_lines_from_input(lines_path)
lines_start = np.array([line.start for line in lines])  # [n, 3]
lines_end = np.array([line.end for line in lines])      # [n, 3]

# 加载点
ply_path = '../outputs/jmsd_with_imgrad/point_cloud/iteration_100000/point_cloud.ply'
original_3dgs = plyfile.PlyData.read(ply_path)
vertices = original_3dgs['vertex']
coords = np.stack([vertices['x'], vertices['y'], vertices['z']], axis=1)
colors = np.stack([vertices['red'], vertices['green'], vertices['blue']], axis=1) / 255.0

# 筛选点
# near_line_mask = []
# for i, coord in enumerate(coords):
#     dist = lines[0].point_distance(coord)
#     threshold = 0.01
#     prob = 0    
#     if dist < threshold:
#         save_mask.append(True)
#     else:
#         if np.random.rand() < prob:
#             save_mask.append(True)
#         save_mask.append(False)
# # new_coords = coords[save_mask]
# # new_colors = colors[save_mask]
# save_mask = np.array(save_mask)
# colors[save_mask] = np.array([1.0, 0.0, 0.0])  # 将选中的点颜色改为红色
# colors[~save_mask] = np.array([0.0, 0.0, 1.0])  # 将未选中的点颜色改为蓝色

# 在线上采样，用于可视化线
lines_points = []
lines_colors = []
for line in lines:
    num_samples = 100
    line_vec = line.end - line.start
    line_points = np.array([line.start + (i / (num_samples - 1)) * line_vec for i in range(num_samples)])
    line_colors = np.tile(np.array([[0.0, 1.0, 0.0]]), (num_samples, 1))  # 绿色
    lines_points.append(line_points)
    lines_colors.append(line_colors)
line_points = np.vstack(lines_points)
line_colors = np.vstack(lines_colors)


I20250729 11:37:19.269258 133708200079680 io.py:read_folder_linetracks:313] Read linetracks from ../../datasets/jmsd/perspective_v1/lines/finaltracks/...


# 显示点和线

In [ ]:
import os
import open3d as o3d

near_line_mask = np.where(min_dist < 0.05, True, False)
new_colors = colors.copy()
new_colors[near_line_mask] = np.array([1.0, 0.0, 0.0])  # 将选中的点颜色改为红色
new_colors[~near_line_mask] = np.array([0.0, 0.0, 1.0])  # 将未选中的点颜色改为蓝色
print(np.where(near_line_mask)[0].shape)
print(np.where(~near_line_mask)[0].shape)

os.environ["XDG_SESSION_TYPE"] = 'x11'
pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(coords)
# pcd.colors = o3d.utility.Vector3dVector(new_colors)
pcd.points = o3d.utility.Vector3dVector(np.vstack([coords, line_points]))
pcd.colors = o3d.utility.Vector3dVector(np.vstack([new_colors, line_colors]))

o3d.visualization.draw_geometries([pcd])

(1580843,)
(3871962,)


# 使用KDTree来进行加速

In [7]:
from scipy.spatial import cKDTree
def batch_point_to_lines_distance_kdtree(points, lines_start, lines_end, device='cpu', k=100, point_batch_size=1024):
    # single patch
    def _batch_point_to_lines_distance_kdtree(points, lines_start, lines_end, tree, k, device):
        dists, idxs = tree.query(points, k=k)  # idxs: [m, k]
        selected_start = lines_start[idxs]  # [m, k, 3]
        selected_end = lines_end[idxs]      # [m, k, 3]
        points_torch = torch.from_numpy(points).float().to(device)         # [m, 3]
        start_torch = torch.from_numpy(selected_start).float().to(device)  # [m, k, 3]
        end_torch = torch.from_numpy(selected_end).float().to(device)      # [m, k, 3]
        P = points_torch.unsqueeze(1)         # [m, 1, 3]
        A = start_torch                       # [m, k, 3]
        B = end_torch                         # [m, k, 3]
        AB = B - A                            # [m, k, 3]
        AP = P - A                            # [m, k, 3]
        t = torch.sum(AP * AB, dim=2) / torch.sum(AB * AB, dim=2)  # [m, k]
        t = torch.clamp(t, 0, 1)
        closest = A + t.unsqueeze(2) * AB     # [m, k, 3]
        dist = torch.norm(P - closest, dim=2) # [m, k]
        min_dist = torch.min(dist, dim=1).values  # [m]
        return min_dist.cpu().numpy()
    
    # Construct KDTree for midpoints of lines
    lines_mid = (lines_start + lines_end) / 2.0  # [n, 3]
    tree = cKDTree(lines_mid)
    # Process points in batches
    m = points.shape[0]
    if m > point_batch_size:
        num_batches = (m + point_batch_size - 1) // point_batch_size
        min_dists = []
        for i in tqdm.tqdm(range(num_batches), desc='Batch processing'):
            start_idx = i * point_batch_size
            end_idx = min((i + 1) * point_batch_size, m)
            batch_points = points[start_idx:end_idx]
            batch_min_dist = _batch_point_to_lines_distance_kdtree(batch_points, lines_start, lines_end, tree, k, device)
            min_dists.append(batch_min_dist)
        return np.concatenate(min_dists, axis=0)
    else:
        return _batch_point_to_lines_distance_kdtree(points, lines_start, lines_end, tree, k, device)

import time
start_time = time.time()
min_dist = batch_point_to_lines_distance_kdtree(coords, lines_start, lines_end, device='cuda', k=100, point_batch_size=16384)
print('with KD-tree: ', time.time() - start_time)
for i in range(10):
    print(f"{i}: {min_dist[i]:.4f}")

Batch processing: 100%|██████████| 333/333 [01:06<00:00,  5.00it/s]

with KD-tree:  66.62541007995605
0: 0.1136
1: 0.3735
2: 0.3047
3: 0.3165
4: 0.1442
5: 15.2494
6: 19.3580
7: 30.5466
8: 28.7358
9: 21.1759


# 使用numba加速

In [5]:
import numpy as np
import numba
from numba import cuda
import math
from time import perf_counter

@cuda.jit
def point_to_line_distance_kernel(points, lines_start, lines_end, min_distances_out):
    """
    这是一个将在 GPU 上运行的 CUDA Kernel。
    每个 GPU 线程将处理一个点。
    """
    # 获取当前 GPU 线程的全局唯一ID
    idx = cuda.grid(1)

    # 边界检查：确保线程ID不会超出点的数组范围
    # (因为我们启动的线程总数可能略多于点的数量)
    if idx >= points.shape[0]:
        return

    point = points[idx]
    min_dist_for_point = 1e10  # 使用一个足够大的数作为初始无穷大

    num_lines = lines_start.shape[0]

    # 每个线程为其分配的点，遍历所有线段
    for i in range(num_lines):
        line_start = lines_start[i]
        line_end = lines_end[i]

        # --- 以下是点到线段距离的三维向量计算 ---
        # 注意：在 @cuda.jit 设备代码中，我们不能使用 numpy 函数 (如 np.dot)
        # 必须使用纯数学运算或 numba 支持的函数。

        # 计算向量
        line_vec_x = line_end[0] - line_start[0]
        line_vec_y = line_end[1] - line_start[1]
        line_vec_z = line_end[2] - line_start[2]

        point_vec_x = point[0] - line_start[0]
        point_vec_y = point[1] - line_start[1]
        point_vec_z = point[2] - line_start[2]
        
        # 点积
        dot_product = point_vec_x * line_vec_x + point_vec_y * line_vec_y + point_vec_z * line_vec_z
        line_len_sq = line_vec_x**2 + line_vec_y**2 + line_vec_z**2
        
        dist_sq = 0.0 # 距离的平方，避免开方

        if line_len_sq < 1e-9: # 线段是一个点
             dist_sq = point_vec_x**2 + point_vec_y**2 + point_vec_z**2
        else:
            t = dot_product / line_len_sq
            if t < 0.0:
                dist_sq = point_vec_x**2 + point_vec_y**2 + point_vec_z**2
            elif t > 1.0:
                dx = point[0] - line_end[0]
                dy = point[1] - line_end[1]
                dz = point[2] - line_end[2]
                dist_sq = dx**2 + dy**2 + dz**2
            else:
                # 投影点
                proj_x = line_start[0] + t * line_vec_x
                proj_y = line_start[1] + t * line_vec_y
                proj_z = line_start[2] + t * line_vec_z
                # 点到投影点的距离平方
                dx = point[0] - proj_x
                dy = point[1] - proj_y
                dz = point[2] - proj_z
                dist_sq = dx**2 + dy**2 + dz**2

        if dist_sq < min_dist_for_point:
            min_dist_for_point = dist_sq

    # 将最终找到的最小距离（开方后）写入输出数组
    min_distances_out[idx] = math.sqrt(min_dist_for_point)


# ===================================================================
# 主机端函数 (这是我们调用的主函数)
# ===================================================================

def find_closest_lines_gpu(points, lines_start, lines_end):
    """
    使用 Numba 和 GPU 并行计算m个点到n条线段的最短距离。

    参数:
    points (np.ndarray): Shape (m, 3) 的点数组。
    lines_start (np.ndarray): Shape (n, 3) 的线段起点数组。
    lines_end (np.ndarray): Shape (n, 3) 的线段终点数组。

    返回:
    np.ndarray: Shape (m,) 的数组，包含每个点到所有线段的最短距离。
    """
    num_points = points.shape[0]
    if num_points == 0:
        return np.array([])
    if lines_start.shape[0] == 0:
        return np.full(num_points, np.inf)

    # 1. 将数据从 CPU 内存 (NumPy arrays) 复制到 GPU 内存
    print("将数据传输到GPU...")
    points_device = cuda.to_device(points)
    lines_start_device = cuda.to_device(lines_start)
    lines_end_device = cuda.to_device(lines_end)
    
    # 2. 在 GPU 上创建一个用于存储结果的空数组
    min_distances_device = cuda.device_array(num_points, dtype=np.float64)

    # 3. 配置 GPU Kernel 的启动参数
    # 设置每个 block 的线程数 (通常是32的倍数，如 128, 256)
    threads_per_block = 1024
    # 计算需要的 block 数量，使用向上取整
    blocks_per_grid = (num_points + (threads_per_block - 1)) // threads_per_block

    print(f"启动GPU Kernel (Grid: {blocks_per_grid}, Block: {threads_per_block})...")
    
    # 4. 启动 Kernel！
    point_to_line_distance_kernel[blocks_per_grid, threads_per_block](
        points_device,
        lines_start_device,
        lines_end_device,
        min_distances_device
    )
    # GPU 现在开始并行计算
    
    # 5. 将计算结果从 GPU 内存复制回 CPU 内存
    print("将结果传回CPU...")
    min_distances_host = min_distances_device.copy_to_host()

    return min_distances_host




# --- 调用核心函数进行计算 ---
try:
    start_time = perf_counter()
    min_dist = find_closest_lines_gpu(coords, lines_start, lines_end)
    end_time = perf_counter()

    # --- 打印结果 ---
    print("\nGPU 计算完成！")
    print(f"总耗时: {end_time - start_time:.4f} 秒")
    for i in range(10):
        print(f"{i}: {min_dist[i]:.4f}")

except numba.cuda.cudadrv.error.CudaSupportError as e:
    print(f"\n错误: 未找到CUDA支持的设备或CUDA工具包未正确安装。")
    print(f"请确保您有NVIDIA GPU并已安装CUDA Toolkit。")
    print(f"Numba 错误信息: {e}")


将数据传输到GPU...
启动GPU Kernel (Grid: 5326, Block: 1024)...
将结果传回CPU...

GPU 计算完成！
总耗时: 76.0321 秒
0: 0.1136
1: 0.3735
2: 0.3047
3: 0.3165
4: 0.1442
5: 15.2494
6: 19.3580
7: 30.5466
8: 26.7938
9: 21.0464


# 使用自己写的fast_point_line_distance

In [5]:
import torch
from fast_point_line_distance import find_shortest_distance
from time import perf_counter
ptsdata = coords
linesdata = np.stack((lines_start, lines_end), axis=1)
start_time = perf_counter()
min_dist = find_shortest_distance(coords, linesdata)
end_time = perf_counter()
print(f"使用自定义函数计算耗时: {end_time - start_time:.4f} 秒")
for i in range(10):
    print(f"{i}: {min_dist[i]:.4f}")

使用自定义函数计算耗时: 0.5668 秒
0: 0.1136
1: 0.3735
2: 0.3047
3: 0.3165
4: 0.1442
5: 15.2494
6: 19.3580
7: 30.5466
8: 26.7938
9: 21.0464


In [13]:
lines[0].as_array()  # 获取线段的数组表示

array([[  6.96874505, -12.91281555,  -1.73939845],
       [  6.94208797, -12.97871864,  -1.13971584]])

# 筛选点

In [6]:
# 筛选点
near_line_mask = np.where(min_dist < 0.01, True, False)
keep_mask = near_line_mask.copy()
new_colors = colors.copy()
false_indices = np.where(~near_line_mask)[0]
random_keep = np.random.rand(false_indices.size) < 0.1
keep_mask[false_indices[random_keep]] = True

new_coords = coords[keep_mask]
new_colors = new_colors[keep_mask]
new_colors[near_line_mask[keep_mask]] = np.array([1.0, 0.0, 0.0])  # True的为红色
new_colors[~near_line_mask[keep_mask]] = np.array([0.0, 0.0, 1.0])


# 可视化筛选结果

In [7]:
import os
import open3d as o3d
os.environ["XDG_SESSION_TYPE"] = 'x11'
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_coords)
pcd.colors = o3d.utility.Vector3dVector(new_colors)

o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 保存ply文件

In [9]:
from plyfile import PlyData, PlyElement
import os

# 用keep_mask筛选所有属性
filtered_vertex = vertices[keep_mask]
# use copy to realocate memory
filtered_vertex = filtered_vertex.copy()

# 构造新的PlyData并保存
filtered_ply = PlyData([PlyElement.describe(filtered_vertex, 'vertex')], text=False)
filtered_ply.write(os.path.join(os.path.dirname(ply_path), 'line_filtered.ply'))
